# Morse Potential

<br>

All information in this Jupyter notebook originates from <i>Diatomic Molecules According to the Wave Mechanics. II. Vibrational Levels</i> by Philip M. Morse.

<br>

The nuclear motion of a diatomic molecule can be described by the following equation: <br>
$$\nabla^2\psi + \frac{8\pi^2\mu}{h^2}[W - E(r)]\psi = 0$$ <br>

The definitions of the variables used in the above equation and throughout this notebook follow below:<br>

$\mu = \frac{M_1M_2}{M_1+M_2}$ <br>
$M_1$ and $M_2$ are respectivly the nuclear masses for nucli 1 and nucli 2. <br>
$Z$ refers to the atomic number of the respective atom.<br>
$r$ refers to the bond distance between two atoms that form a diatomic molecule.<br>
$V_e(r)$ refers to the electronic energy of the system.<br>
$E(r) = \frac{e^2Z_1Z_2}{r} - V_e(r)$ is the nuclear potential energy equation.

&nbsp;&nbsp;&nbsp;&nbsp; $E(r)$ can be considered to be the sum of the electronic energy and the nuclear repulsions. The exact form of this equation is unknown due to its non-trival relation with the electronic quantum numbers and $r$, thus approximations are used instead of the exact $E(r)$. The approximations used must satisfy the following conditions, which are directly quoted from Morse's paper. <br>

1. The function should come asymptotically to a finite value as $r\rightarrow \infty$
2. The function should have its only minumum point at $r=r_0$
3. The function should become infinity, or very large, at $r=0$
4. The function should give the allowed energy levels equivalent to the finite polynomial:
$$ W(n) = -D + h\omega_0[(n+\frac{1}{2})-x(n+\frac{1}{2})^2] $$

The above finite polynomial is derived from two series used to approximate $E(r)$, and the function proposed by Morse that satisfies the above condtions is the following:

$$ E(r)=De^{-2a(r-r_0)}-2De^{-a(r-r_0)} $$

Which, with algebraic manipulation, can also be written as:

$$ E(r) = D(1-e^{(-a(r-r_0))^2} $$ 

Below follows definitions for variables used in the Morse Potential: <br>
$D$ is the well depth for the diatomic molecule's nuclear potential energy. <br>
$r_0$ is the optimal diatomic bond distance at which the nuclear potential energy is at a minimum.<br>
$\omega_0$ is the frequency of classical vibrations at $r_0$ <br>
$a$ is related to $\omega_0$ by $\omega_0 = \frac{a}{2\pi}(\frac{2D}{\mu})^{1/2}$, which can also be expressed as $a=\sqrt{\frac{2\pi\omega_0^2\mu}{D}}$ <br>

In [1]:
#Import Packages Here
import math
from plotly.offline import iplot, init_notebook_mode

##connect plotly to the Juypter notebook
init_notebook_mode(connected=True)

In [2]:
#Declare all variables used for the computation here

#All User Input Variables Defined Here
#Nucli Masses in Atomic Units
M1 = 1
M2 = 1

#Atomic Numbers
Z1 = 1
Z2 = 1

#Enter Well Depth for the molecule as a negative value
D = 1.1246043918275317

#Enter optimal bond distance, the distance at which the well depth is at its minimum
r0 = 1.41

#Enter the classical vibrational frequency of the molecule at r0
w = 0.16629000000002617

#Enter the starting and ending r values
startR = 0
endR = 5

#Enter a delta R to increment r by the specified amount for graphing purposes
dr = 0.001

#All calculated variables declared here
u = (M1*M2) / (M1+M2)
a = 0.49624511718750053#math.sqrt( 2 * pow(math.pi*w,2) * u / D )
r = startR

E = []
rData = []

#calculate the approximate E(r) values using the Morse Potential
while(r <= endR):
    rData.append(r)
    
    E.append( 
        #Morse Potential Equation
        (D * math.exp(-2 * a * ( r- r0 )) - (2 * D * math.exp(-a * ( r - r0 ))))
    )
    
    r += dr

#Prepare the graph data for plotly here
figure = {"data": [
        
        #create the Morse Potential Graph
        {
            "type":"scatter",
            "x":rData,
            "y":E,
            "connectgaps":True,
            "mode":"markers", 
            "name":"Morse Potential",
            "marker":{"color":"blue"}
        },
    
        #highlight the minimum energy point in red
        {
            "type":"scatter",
            "x":[r0],
            "y":[-D],
            "name":"Actual Energy Minimum",
            "marker":{"color":"red"}
        },
        
        #Highlight the mminimum bond point highlighted by the Morse Potential
        {
            "type":"scatter",
            "x":[rData[E.index(min(E))]],
            "y":[min(E)],
            "name":"Predicted Minimum",
            "marker":{"color":"yellow"}
        },
    ],
    "layout":
        {
           "xaxis":{"title":"Bond Distance"},
           "yaxis":{"title":"E(r)"},
            "title":{"text":"Morse Potential Graph"}
        },    
}

iplot(figure)

In [3]:
#Morse Potential Basis Testing
import math
import numpy as np
from scipy.integrate import quad as integrate

def hermite(n, k):
    
    c = pow(-1, k) * math.factorial(n) / ( math.factorial(k) * math.factorial(n - 2*k) )
        
    if(k == 0):
        return lambda r : c * pow(2*(r), n-(2*k))
    else:
        return lambda r : c * pow(2*(r), n-(2*k)) + hermite(n, k-1)(r)

def buildHermite(n):    
    return hermite(n, n // 2)
#returns the normalization constant for the Harmonic Oscillator Function
def C(n):
    return math.sqrt( 1 /  (pow(2, n) * math.factorial(n) * math.sqrt(math.pi)) ) 

#Creates a new Harmonic Oscillator function
#and returns it as a lambda function
def newHO(n):
    return lambda r : C(n) * buildHermite(n)(r) * math.exp(-pow(r,2) / 2)

psi0 = newHO(0)

ans = integrate( lambda x : psi0(x) * psi0(x), -100, 100)

print(ans)

h = buildHermite(0)
#Prepare the graph data for plotly here
figure = {"data": [
        
        #create the Morse Potential Graph
        {
            "type":"scatter",
            "x":list(range(-20,20)),
            "y":[h(x) for x in range(-20,20)],
            "connectgaps":True,
            "mode":"markers", 
            "name":"H0",
            "marker":{"color":"blue"}
        },
     #create the Morse Potential Graph
        {
            "type":"scatter",
            "x":list(range(-10,10)),
            "y":[psi0(x) for x in range(-5,5)],
            "connectgaps":True,
            "mode":"markers", 
            "name":"psi0",
            "marker":{"color":"red"}
        },
    ],      
}

iplot(figure)


(1.0000000000000002, 1.1152971710910976e-10)
